In [1]:
import functools
import numpy as np
import tensorflow as tf
from keras.datasets import cifar100
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.utils import to_categorical
from keras.utils.generic_utils import get_custom_objects
from keras.initializers import RandomNormal, GlorotNormal

In [2]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2)

In [3]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [4]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

((40000, 32, 32, 3),
 (40000, 100),
 (10000, 32, 32, 3),
 (10000, 100),
 (10000, 32, 32, 3),
 (10000, 100))

In [5]:
train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)

train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [6]:
def simplenet(act, s = 2):
    model = Sequential()

    model.add(Conv2D(64, (3,3), padding='same', kernel_initializer=GlorotNormal(), input_shape=(32, 32, 3)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(256, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(256, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(256, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    
    model.add(Conv2D(512, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))

    model.add(Conv2D(2048, (1,1), padding='same', kernel_initializer=GlorotNormal()))
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(256, (1,1), padding='same', kernel_initializer=GlorotNormal()))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(Dropout(0.2))

    model.add(Conv2D(256, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))

    model.add(Flatten())
    model.add(Dense(100, activation='softmax'))

    return model

In [19]:
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

def swish(x):
    return x * tf.sigmoid(x)
get_custom_objects().update({'swish': Activation(swish)})

get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

# act_func = ['tanh', 'relu', 'leaky-relu', 'elu', 'selu', 'gelu', 'swish']

model = simplenet('swish')

batch_size = 128
epochs = 50

top3_acc = functools.partial(tf.keras.metrics.top_k_categorical_accuracy, k=3)

top3_acc.__name__ = 'top3_acc'

model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy', top3_acc, 'top_k_categorical_accuracy'])

history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
                      validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), verbose=1)

Epoch 1/50
313/313 [==============================] - 52s 154ms/step - loss: 4.5683 - accuracy: 0.0142 - top3_acc: 0.0411 - top_k_categorical_accuracy: 0.0682 - val_loss: 4.6046 - val_accuracy: 0.0113 - val_top3_acc: 0.0337 - val_top_k_categorical_accuracy: 0.0534
Epoch 2/50
313/313 [==============================] - 46s 146ms/step - loss: 4.4713 - accuracy: 0.0269 - top3_acc: 0.0694 - top_k_categorical_accuracy: 0.1050 - val_loss: 4.5968 - val_accuracy: 0.0175 - val_top3_acc: 0.0424 - val_top_k_categorical_accuracy: 0.0638
Epoch 3/50
313/313 [==============================] - 46s 148ms/step - loss: 4.4037 - accuracy: 0.0391 - top3_acc: 0.0946 - top_k_categorical_accuracy: 0.1362 - val_loss: 4.5816 - val_accuracy: 0.0231 - val_top3_acc: 0.0521 - val_top_k_categorical_accuracy: 0.0737
Epoch 4/50
313/313 [==============================] - 46s 148ms/step - loss: 4.3419 - accuracy: 0.0487 - top3_acc: 0.1148 - top_k_categorical_accuracy: 0.1652 - val_loss: 4.5825 - val_accuracy: 0.0187 - va

In [20]:
print(history.history)
# y_pred = np.argmax(model.predict(x_test), axis=1)
# y_true = np.argmax(y_test,axis=1)

# print(y_pred.shape)
# print(y_true.shape)

# print(np.sum(y_pred == y_true) / y_pred.shape[0])

print(model.evaluate(x_test, y_test))

{'loss': [4.532835006713867, 4.458946228027344, 4.3915228843688965, 4.315042018890381, 4.219127655029297, 4.054031848907471, 3.861006736755371, 3.671999216079712, 3.512974500656128, 3.366607427597046, 3.2253758907318115, 3.08965802192688, 2.9496653079986572, 2.8157145977020264, 2.6885876655578613, 2.585306406021118, 2.4799907207489014, 2.404360055923462, 2.32139253616333, 2.2495954036712646, 2.1831934452056885, 2.120950937271118, 2.058474540710449, 2.012575149536133, 1.9602528810501099, 1.9125357866287231, 1.8696393966674805, 1.8389382362365723, 1.791387677192688, 1.756760597229004, 1.7137351036071777, 1.6813251972198486, 1.6512950658798218, 1.6239362955093384, 1.5925995111465454, 1.5673080682754517, 1.5403305292129517, 1.5145268440246582, 1.4858486652374268, 1.463504672050476, 1.4435319900512695, 1.4235827922821045, 1.3978415727615356, 1.375935673713684, 1.3650635480880737, 1.3284506797790527, 1.3238279819488525, 1.2936878204345703, 1.2796080112457275, 1.2530425786972046], 'accuracy':